<a href="https://colab.research.google.com/github/srikanth-gedela/SriMLModels/blob/main/Pdf2Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytesseract;
!pip install PyPDF2;
!pip install wand;

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!apt-get install libmagickwand-dev
!apt-get install libjpeg-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libmagickwand-dev is already the newest version (8:6.9.10.23+dfsg-2.1ubuntu11.4).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libjpeg-dev is already the newest version (8c-2ubuntu8).
libjpeg-dev set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.


In [ ]:
import PyPDF2
import pytesseract
import wand
from PIL import Image
from wand.image import Image as WandImage
import io
from wand.exceptions import MissingDelegateError



In [ ]:

# Function to convert PDF to text
def pdf_to_text(file_path, start_page=None, end_page=None):
    # Open the PDF file
    pdf_file = open(file_path, 'rb')

    # Create a PDF reader object
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    # If start and end page numbers are not provided, convert all pages
    if start_page is None:
        start_page = 0
    if end_page is None:
        end_page = pdf_reader.numPages

    # Create an empty string to store the text
    text = ""

    # Loop through the pages and extract the text
    for page_num in range(start_page, end_page):
        page = pdf_reader.pages[page_num] 

        # Check if the page is empty
        if page['/Contents'] is None:
            continue

        # Check if the page is searchable or requires OCR
        if page.extract_text():
            text += page.extract_text()
        else:
            # Convert the page to an image and perform OCR
            with io.BytesIO() as byte_stream:
                try:
                    with WandImage(file=pdf_file, resolution=300) as wand_image:
                        wand_image.compression_quality = 99
                        wand_image.format = 'jpeg'
                        wand_image.background_color = 'white'
                        wand_image.alpha_channel = 'remove'
                        wand_image.save(byte_stream)
                    byte_stream.seek(0)
                    text += pytesseract.image_to_string(byte_stream, lang='eng')
                except MissingDelegateError as e:
                    print(f"Failed to convert page {page_num}: {str(e)}")

    # Close the PDF file
    pdf_file.close()

    return text



In [ ]:

# Function to split text into multiple files
def split_text(text, max_words_per_file=20000):
    # Split the text into a list of words
    words = text.split()

    # Create an empty list to store the file names
    file_names = []

    # Loop through the words and create a new file every max_words_per_file words
    for i in range(0, len(words), max_words_per_file):
        file_name = f"output_{i//max_words_per_file}.txt"
        file_names.append(file_name)
        with open(file_name, 'w') as file:
            file.write(' '.join(words[i:i+max_words_per_file]))

    return file_names



In [ ]:

# Example usage
file_path = "input.pdf"
start_page = 17
end_page = 173
max_words_per_file = 20000

text = pdf_to_text(file_path, start_page, end_page)
file_names = split_text(text)

print(f"Input file converted to {len(file_names)} output files of {max_words_per_file} words each.")

Failed to convert page 25: no decode delegate for this image format `' @ error/blob.c/BlobToImage/361
Failed to convert page 53: no decode delegate for this image format `' @ error/blob.c/BlobToImage/361
Failed to convert page 117: no decode delegate for this image format `' @ error/blob.c/BlobToImage/361
Failed to convert page 139: no decode delegate for this image format `' @ error/blob.c/BlobToImage/361
Failed to convert page 141: no decode delegate for this image format `' @ error/blob.c/BlobToImage/361
Failed to convert page 147: no decode delegate for this image format `' @ error/blob.c/BlobToImage/361
Failed to convert page 159: no decode delegate for this image format `' @ error/blob.c/BlobToImage/361
Failed to convert page 171: no decode delegate for this image format `' @ error/blob.c/BlobToImage/361
Input file converted to 3 output files of 20000 words each.
